[Reference](https://medium.com/thedeephub/predict-food-demand-with-python-part-2-2ede9b36e9d0)

In [14]:
!pip install catboost
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 7.7 MB/s eta 0:00:00


In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from scipy import stats

import matplotlib.dates as mdates
%matplotlib inline

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from category_encoders.binary import BinaryEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, ReLU, Bidirectional
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error

from sklearn.preprocessing import QuantileTransformer, PowerTransformer
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings("ignore")

# Mounting Google Drive
from google.colab import drive  # Import for accessing Google Drive

# Unzipping files
import zipfile  # Import for extracting zip files

# Read data

In [16]:
# Mount your Google Drive
drive.mount('/content/drive')

# Get the file path from Google Drive
file_path = '/content/drive/MyDrive/Colab Notebooks/datasets/Forecasting_of_Food_Demand.zip'

# Unzip the file
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    # Extract the CSV files
    train_path = zip_ref.extract('train.csv', '/content/')
    test_path = zip_ref.extract('test.csv', '/content/')
    sub_path = zip_ref.extract('sample_submission.csv', '/content/')
    meals_path = zip_ref.extract('meal_info.csv', '/content/')
    centres_path = zip_ref.extract('fulfilment_center_info.csv', '/content/')

# Read the CSV files
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
sub = pd.read_csv(sub_path)
meals = pd.read_csv(meals_path)
centres = pd.read_csv(centres_path)

# Features Scaling and Encoding

In [1]:
train_week = train[['week']]

categoric_columns = ['center_id', 'meal_id', 'emailer_for_promotion', 'homepage_featured', 'city_code', 'region_code', 'center_type', 'op_area', 'category', 'cuisine']
columns = list(train.columns)
numeric_columns = [i for i in columns if i not in categoric_columns]

numeric_columns.remove('num_orders')

In [2]:
encoder = BinaryEncoder(drop_invariant=False, return_df=True,)
encoder.fit(train[categoric_columns])

In [3]:
scaler = StandardScaler()
scaler.set_output(transform="pandas")
quantile_transformer = QuantileTransformer(output_distribution='normal')
train_num_quantile = quantile_transformer.fit_transform(train[numeric_columns])
scaler.fit(train_num_quantile)

In [4]:
encoded_cat = encoder.transform(train[categoric_columns])

scaled_num = scaler.transform(train[numeric_columns])
# encoded_cat = train[categoric_columns].apply(encoder.fit_transform)

train = pd.concat([scaled_num, encoded_cat, train.num_orders], axis=1)

In [5]:
train['week_unscaled'] = train_week
# Split the dataset into training (weeks 1-135) and evaluation (weeks 136-145) sets
trainn = train[train['week_unscaled'] <= 135]
evall = train[train['week_unscaled'] > 135]

# Display the shapes of the training and evaluation sets
print("Training set shape:", trainn.shape)
print("Evaluation set shape:", evall.shape)

trainn.drop('week_unscaled', axis=1, inplace=True)
evall.drop('week_unscaled', axis=1, inplace=True)

In [17]:
# Split data into parts
X_train = trainn.drop(['num_orders'], axis = 1)
X_test = evall.drop(['num_orders'], axis = 1)
y_train = trainn['num_orders']
y_test = evall['num_orders']

# ML Models for Forecasting

In [18]:
# Define the parameter grid
param_grid = {
    'max_depth': [8, 9, 10],
    'max_features': ['sqrt'],
    'n_estimators': [100, 150, 200],
    'min_samples_leaf': [2, 3, 4]
}

# Initialize the Random Forest Regressor
forest = RandomForestRegressor()
model_forest = forest.fit(X_train, y_train)
# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=forest, param_grid=param_grid, scoring='neg_mean_squared_log_error', cv=5)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters and best RMSLE score
best_params = grid_search.best_params_
best_rmsle = np.sqrt(-grid_search.best_score_)

# Print the best parameters and best RMSLE score
print("Best Parameters:", best_params)
print("Best RMSLE Score:", best_rmsle)

In [7]:
# Define the parameter grid
param_grid = {
    'max_depth': [8, 9, 10],
    'max_features': ['sqrt'],
    'n_estimators': [100, 150, 200],
    'min_samples_leaf': [2, 3, 4]
}

# Initialize the Random Forest Regressor
forest = RandomForestRegressor()

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=forest, param_grid=param_grid, scoring='neg_mean_squared_log_error', cv=5)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters and best RMSLE score
best_params = grid_search.best_params_
best_rmsle = np.sqrt(-grid_search.best_score_)

# Print the best parameters and best RMSLE score
print("Best Parameters:", best_params)
print("Best RMSLE Score:", best_rmsle)

In [19]:
# Gradient Boosting Machine
# Initialize and fit the Gradient Boosting model
gbr = GradientBoostingRegressor()
model_gbr = gbr.fit(X_train, y_train)

# Make predictions on X_test
gbr_pred = model_gbr.predict(X_test)

# Light GBM
# Initialize and fit the LightGBM Regressor
lgbm = lgb.LGBMRegressor()
model_lgbm = lgbm.fit(X_train, y_train)

# XGBoost
# Initialize and fit the XGBoost model
xgboost = xgb.XGBRegressor()
model_xgboost = xgboost.fit(X_train, y_train)

# CatBoost Regressor
# Initialize and fit the CatBoost Regressor
catboost = cb.CatBoostRegressor(silent=True)
model_catboost = catboost.fit(X_train, y_train)

In [8]:
forest_pred = model_forest.predict(X_test)
mse = mean_squared_error(y_test, forest_pred)
msle = mean_squared_log_error(y_test, forest_pred)
rmse = np.sqrt(mse).round(2)
rmsle = np.sqrt(msle).round(5)

# Append the results to the DataFrame
results = pd.DataFrame([['Random Forest', mse, msle, rmse, rmsle]],
                             columns=['Model', 'MSE', 'MSLE', 'RMSE', 'RMSLE'])

gbr_pred = model_gbr.predict(X_test)
gbr_pred = np.abs(gbr_pred)
# Append the results to the DataFrame
mse = mean_squared_error(y_test, gbr_pred)
msle = mean_squared_log_error(y_test, gbr_pred)
rmse = np.sqrt(mse).round(2)
rmsle = np.sqrt(msle).round(5)

model_results = pd.DataFrame([['Gradient Boosting', mse, msle, rmse, rmsle]],
                             columns=['Model', 'MSE', 'MSLE', 'RMSE', 'RMSLE'])
results = pd.concat([results, model_results], ignore_index=True)

lgbm_pred = np.abs(model_lgbm.predict(X_test))

# Compute performance metrics
mse = mean_squared_error(y_test, lgbm_pred)
msle = mean_squared_log_error(y_test, lgbm_pred)
rmse = np.sqrt(mse).round(2)
rmsle = np.sqrt(msle).round(5)

# Create a DataFrame for the model results
model_results = pd.DataFrame([['LightGBM', mse, msle, rmse, rmsle]],
                             columns=['Model', 'MSE', 'MSLE', 'RMSE', 'RMSLE'])

# Concatenate the new results to the existing results DataFrame
results = pd.concat([results, model_results], ignore_index=True)

xgboost_pred = np.abs(model_xgboost.predict(X_test))

# Append the results to the DataFrame
mse = mean_squared_error(y_test, xgboost_pred)
msle = mean_squared_log_error(y_test, xgboost_pred)
rmse = np.sqrt(mse).round(2)
rmsle = np.sqrt(msle).round(5)

model_results = pd.DataFrame([['XGBoost', mse, msle, rmse, rmsle]],
                             columns=['Model', 'MSE', 'MSLE', 'RMSE', 'RMSLE'])
results = pd.concat([results, model_results], ignore_index=True)

catboost_pred = np.abs(model_catboost.predict(X_test))

# Compute performance metrics
mse = mean_squared_error(y_test, catboost_pred)
msle = mean_squared_log_error(y_test, catboost_pred)
rmse = np.sqrt(mse).round(2)
rmsle = np.sqrt(msle).round(5)

# Create a DataFrame for the model results
model_results = pd.DataFrame([['CatBoost', mse, msle, rmse, rmsle]],
                             columns=['Model', 'MSE', 'MSLE', 'RMSE', 'RMSLE'])

# Concatenate the new results to the existing results DataFrame
results = pd.concat([results, model_results], ignore_index=True)
results

In [9]:
# Create sequences for LSTM input
def create_sequences(X, y, time_steps=10):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X.iloc[i:(i + time_steps)].values)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

time_steps = 10
X_train_seq, y_train_seq = create_sequences(X_train, y_train, time_steps)
X_test_seq, y_test_seq = create_sequences(X_test, y_test, time_steps)

# Reshape y_train_seq and y_test_seq to be 2D arrays
y_train_seq = y_train_seq.reshape(-1, 1)
y_test_seq = y_test_seq.reshape(-1, 1)

# Check the shapes
print(X_train_seq.shape, y_train_seq.shape)
print(X_test_seq.shape, y_test_seq.shape)

# Now proceed to create and train the LSTM model

# Define the LSTM model based on the provided architecture
def create_lstm_model(input_shape):
    model = Sequential()

    # LSTM layer 1
    model.add(LSTM(64, input_shape=input_shape, return_sequences=True))
    model.add(ReLU())
    model.add(Dropout(0.25))

    # LSTM layer 2
    model.add(LSTM(32, return_sequences=True))
    model.add(ReLU())
    model.add(Dropout(0.25))

    # LSTM layer 3
    model.add(LSTM(16))
    model.add(ReLU())
    model.add(Dropout(0.25))

    # Dense layer
    model.add(Dense(1))

    return model

# Define the Bi-LSTM model
def create_bilstm_model(input_shape):
    model = Sequential()

    # Bi-LSTM layer 1
    model.add(Bidirectional(LSTM(32, return_sequences=True, dropout=0.25, recurrent_activation='tanh'), input_shape=input_shape))

    # Bi-LSTM layer 2
    model.add(Bidirectional(LSTM(16, return_sequences=False, dropout=0.25, recurrent_activation='tanh')))

    # Dense layer
    model.add(Dense(1))

    return model